# This notebook serves as walkthrough for planning an experiment for creation through the OT2.
### The following modules are used and should be in the same directory as this notebook: 
* **CreateSamples** is responsible for calculating sample information, which includes component weight fractions and stock volumes
* **OT2Commands** is responsible for setting up information to be interpretted and executed by opentrons **BASED ON THE LATEST API 2.3 and above**
* **OT2Graphing** contains graphing tools to help visualize and explore parameter spaces.

In [1]:
from Plan import CreateSamples
from Prepare import OT2Commands as ALH
# from Prepare import OT2Graphing as ographing
from opentrons import simulate, execute, protocol_api
import importlib
import pandas as pd
import time 
import numpy as np

In [2]:
importlib.reload(ALH)
importlib.reload(CreateSamples)

<module 'Plan.CreateSamples' from 'C:\\Users\\pozzo_admin\\Desktop\\research\\packages\\OT2-DOE\\PlanPrepareProcess\\Plan\\CreateSamples.py'>

In [3]:
def get_pipette_tip_count(small_pipette, large_pipette, volume_df):
    
    small_tips = ((volume_df < small_pipette)  & (volume_df !=0)).sum().sum()
    small_racks = np.round(small_tips/96, 1)
    print(" \033[1mSmall\033[0m pipette tips: \033[1m{}\033[0m". format(small_tips))
    print(" \033[1mSmall\033[0m tipracks: \033[1m{}\033[0m". format(small_racks))
    
    
    large_tips = ((volume_df > small_pipette)).sum().sum()
    large_racks = np.round(large_tips/96, 2)
    print(" \033[1mLarge\033[0m pipette tips: \033[1m{}\033[0m". format(large_tips))
    print(" \033[1mLarge\033[0m tipracks: \033[1m{}\033[0m". format(large_racks))

Define two dictionaries to keep track of the species involved in this experiment. 
For the DES Campaign we have 10 Quaternary Ammonium Salts (QAS) and 10 Hydrogen Bond Donors (HBD). 


In [4]:
HBD_dict= {'A1': 'Ethylene Glycol',
           'A2':'Glycerol',
           'A3' : 'Acetamide',
           'A4': 'N,N-dimethylurea',
           'A5': '3-phenylpropionic acid',
           'A6': 'Urea',
           'A7': 'L-serine',
           'A8': '4-amino triazole',
           'A9': 'Xylitol',
           'A10': 'Phenylacetic acid'}
QAS_dict= {'A1': 'Acetylcholine Chloride',
           'A2':'Tetraethylammonium Chloride',
           'A3' : 'Tetrapropylammonium Bromide',
           'A4': 'Benzyltriethylammonium Bromide',
           'A5': '(1-butyl) triethylammonium Bromide',
           'A6': 'Hexamethonium Bromide',
           'A7': 'Tetraethylammonium Iodide',
           'A8': 'Trimethylpropylammonium Bromide',
           'A9': 'Ethylammonium Chloride',
           'A10': '2-hydroxyethylammonium Chloride'}

There is a total of 100 different combinations of species. 

We plan on formulating 12 different mole fractions between each combination of QAS and HBD. 

The required volumes of each components, based on their stock solutions concentration and the total sampple volume (600uL), was calcuated using a different set of equations generated by Jaime Rodriguez.  

The code below uses the functions of the OT2-DOE GitHub repo to run the campaign. 

In [5]:
volume_data = pd.read_csv('./final_volume_table.csv', index_col=0)

In [6]:
volume_data

,Acetylcholine Chloride-stock,Tetraethylammonium Chloride-stock,Tetrapropylammonium Bromide-stock,Benzyltriethylammonium Bromide-stock,(1-butyl) triethylammonium Bromide-stock,Hexamethonium Bromide-stock,Tetraethylammonium Iodide-stock,Trimethylpropylammonium Bromide-stock,Ethylammonium Chloride-stock,2-hydroxyethylammonium Chloride-stock,Ethylene Glycol-stock,Glycerol-stock,Acetamide-stock,"N,N-dimethylurea-stock",3-phenylpropionic acid-stock,Urea-stock,L-serine-stock,4-amino triazole-stock,Xylitol-stock,Phenylacetic acid-stock
0,97.674419,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,502.325581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,182.608696,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,417.391304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,257.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,342.857143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,291.089109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,308.910891,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,323.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,276.923077,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,206.250000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,393.750000
1196,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,234.782609,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,365.217391
1197,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,300.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,300.000000
1198,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,378.947368,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,221.052632


The total stock volumes required for the entire campaign is as follows: 

`Note: the volume units displayed are mL`

In [7]:
stock_volumes = CreateSamples.isolate_common_column(volume_data,'stock').sum(axis=0)/1000
print( 'Overall Volumes required for the entire campaign:')
stock_volumes

Overall Volumes required for the entire campaign:


Acetylcholine Chloride-stock                36.881370
Tetraethylammonium Chloride-stock           40.840633
Tetrapropylammonium Bromide-stock           46.241007
Benzyltriethylammonium Bromide-stock        46.241007
(1-butyl) triethylammonium Bromide-stock    40.840633
Hexamethonium Bromide-stock                 54.464040
Tetraethylammonium Iodide-stock             54.464040
Trimethylpropylammonium Bromide-stock       40.840633
Ethylammonium Chloride-stock                29.224815
2-hydroxyethylammonium Chloride-stock       29.224815
Ethylene Glycol-stock                       24.375413
Glycerol-stock                              24.375413
Acetamide-stock                             24.375413
N,N-dimethylurea-stock                      24.375413
3-phenylpropionic acid-stock                35.458607
Urea-stock                                  24.375413
L-serine-stock                              40.887430
4-amino triazole-stock                      31.596691
Xylitol-stock               

Since the robot has a limited space for labware, stock reservoirs and pipette racks, the campaign will have to be split over different runs. 

**Each run will involve 2 different QAS and all their combinations with the 10 HBD species**

In [8]:
def get_experimental_subset( QAS1, QAS2, volume_dataframe):
    
    if QAS1.endswith('-stock'):
        pass
    else:
        QAS1 = QAS1 +'-stock'
        
    if QAS2.endswith('-stock'):
        pass
    else:
        QAS2 = QAS2 +'-stock'  
        
    subset_df = volume_dataframe[(volume_dataframe[QAS1] != 0) | (volume_dataframe[QAS2]!= 0)]
    subset_df=subset_df.loc[:,(subset_df !=0).any(axis=0)]
    
    return subset_df
    

In [9]:
run1= get_experimental_subset(QAS_dict['A1'], QAS_dict['A2'], volume_data)
run2= get_experimental_subset(QAS_dict['A3'], QAS_dict['A4'], volume_data)
run3= get_experimental_subset(QAS_dict['A5'], QAS_dict['A6'], volume_data)
run4= get_experimental_subset(QAS_dict['A7'], QAS_dict['A8'], volume_data)
run5= get_experimental_subset(QAS_dict['A9'], QAS_dict['A10'], volume_data)

In [10]:
run1

,Acetylcholine Chloride-stock,Tetraethylammonium Chloride-stock,Ethylene Glycol-stock,Glycerol-stock,Acetamide-stock,"N,N-dimethylurea-stock",3-phenylpropionic acid-stock,Urea-stock,L-serine-stock,4-amino triazole-stock,Xylitol-stock,Phenylacetic acid-stock
0,97.674419,0.0,502.325581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,182.608696,0.0,417.391304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,257.142857,0.0,342.857143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,291.089109,0.0,308.910891,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,323.076923,0.0,276.923077,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
235,0.000000,330.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,270.0
236,0.000000,360.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,240.0
237,0.000000,420.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,180.0
238,0.000000,480.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,120.0


In [11]:
volumes_run1 = CreateSamples.isolate_common_column(run1,'stock').sum(axis=0)/1000
print('Total volume required for subset 1:')
volumes_run1

Total volume required for subset 1:


Acetylcholine Chloride-stock         36.881370
Tetraethylammonium Chloride-stock    40.840633
Ethylene Glycol-stock                 5.408743
Glycerol-stock                        5.408743
Acetamide-stock                       5.408743
N,N-dimethylurea-stock                5.408743
3-phenylpropionic acid-stock          7.786246
Urea-stock                            5.408743
L-serine-stock                        8.905139
4-amino triazole-stock                6.970405
Xylitol-stock                         7.786246
Phenylacetic acid-stock               7.786246
dtype: float64

In [13]:
importlib.reload(ALH)

<module 'Prepare.OT2Commands' from 'C:\\Users\\pozzo_admin\\Desktop\\research\\packages\\OT2-DOE\\PlanPrepareProcess\\Prepare\\OT2Commands.py'>

In [14]:
ALH.calculate_total_volumes(run1)

Acetylcholine Chloride-stock         36.881370
Tetraethylammonium Chloride-stock    40.840633
Ethylene Glycol-stock                 5.408743
Glycerol-stock                        5.408743
Acetamide-stock                       5.408743
N,N-dimethylurea-stock                5.408743
3-phenylpropionic acid-stock          7.786246
Urea-stock                            5.408743
L-serine-stock                        8.905139
4-amino triazole-stock                6.970405
Xylitol-stock                         7.786246
Phenylacetic acid-stock               7.786246
dtype: float64

In [42]:
volumes_run2 = CreateSamples.isolate_common_column(run2,'stock').sum(axis=0)/1000
print('Total volume required for subset 2:')
volumes_run2

Total volume required for subset 2:


Tetrapropylammonium Bromide-stock       46.241007
Benzyltriethylammonium Bromide-stock    46.241007
Ethylene Glycol-stock                    4.004455
Glycerol-stock                           4.004455
Acetamide-stock                          4.004455
N,N-dimethylurea-stock                   4.004455
3-phenylpropionic acid-stock             6.227953
Urea-stock                               4.004455
L-serine-stock                           7.380000
4-amino triazole-stock                   5.431851
Xylitol-stock                            6.227953
Phenylacetic acid-stock                  6.227953
dtype: float64

In [43]:
volumes_run3 = CreateSamples.isolate_common_column(run3,'stock').sum(axis=0)/1000
print('Total volume required for subset 3:')
volumes_run3

Total volume required for subset 3:


(1-butyl) triethylammonium Bromide-stock    40.840633
Hexamethonium Bromide-stock                 54.464040
Ethylene Glycol-stock                        3.791107
Glycerol-stock                               3.791107
Acetamide-stock                              3.791107
N,N-dimethylurea-stock                       3.791107
3-phenylpropionic acid-stock                 5.878698
Urea-stock                                   3.791107
L-serine-stock                               6.975373
4-amino triazole-stock                       5.128324
Xylitol-stock                                5.878698
Phenylacetic acid-stock                      5.878698
dtype: float64

In [44]:
volumes_run4 = CreateSamples.isolate_common_column(run4,'stock').sum(axis=0)/1000
print('Total volume required for subset 4:')
volumes_run4

Total volume required for subset 4:


Tetraethylammonium Iodide-stock          54.464040
Trimethylpropylammonium Bromide-stock    40.840633
Ethylene Glycol-stock                     3.791107
Glycerol-stock                            3.791107
Acetamide-stock                           3.791107
N,N-dimethylurea-stock                    3.791107
3-phenylpropionic acid-stock              5.878698
Urea-stock                                3.791107
L-serine-stock                            6.975373
4-amino triazole-stock                    5.128324
Xylitol-stock                             5.878698
Phenylacetic acid-stock                   5.878698
dtype: float64

In [45]:
volumes_run5 = CreateSamples.isolate_common_column(run5,'stock').sum(axis=0)/1000
print('Total volume required for subset 5:')
volumes_run5

Total volume required for subset 5:


Ethylammonium Chloride-stock             29.224815
2-hydroxyethylammonium Chloride-stock    29.224815
Ethylene Glycol-stock                     7.380000
Glycerol-stock                            7.380000
Acetamide-stock                           7.380000
N,N-dimethylurea-stock                    7.380000
3-phenylpropionic acid-stock              9.687012
Urea-stock                                7.380000
L-serine-stock                           10.651545
4-amino triazole-stock                    8.937789
Xylitol-stock                             9.687012
Phenylacetic acid-stock                   9.687012
dtype: float64

### Check for Pipette tips to use

Each of the QAS selected per run will be distributed using a single pipette, as there is no other sample in the vials. Therefore, the two species will no be included in the pipette count for the experiment. 

We will assume that 2 small pipette tips and 2 large pipette tips will be used for their experiment- will need to double check this!! 

In [35]:
HBD_df_1 = run1.drop([QAS_dict['A1']+'-stock',QAS_dict['A2']+'-stock'], axis=1)
## the first 2 arguments correspond to the maximum volume a pipette can move expressed in uL
print('Run1 \n{} & {} \n'.format(QAS_dict['A1'],QAS_dict['A2']))
get_pipette_tip_count(300,1000, HBD_df_1)

Run1 
Acetylcholine Chloride & Tetraethylammonium Chloride 

 Small pipette tips: 135
 Small tipracks: 1.4
 Large pipette tips: 101
 Large tipracks: 1.05


In [14]:
HBD_df_2 = run2.drop([QAS_dict['A3']+'-stock',QAS_dict['A4']+'-stock'], axis=1)
## the first 2 arguments correspond to the maximum volume a pipette can move expressed in uL
print('Run2 \n{} & {} \n'.format(QAS_dict['A3'],QAS_dict['A4']))
get_pipette_tip_count(300,1000, HBD_df_2)

 Small pipette tips: 176
 Small tipracks: 1.8
 Large pipette tips: 62
 Large tipracks: 0.65


In [18]:
HBD_df_3 = run3.drop([QAS_dict['A5']+'-stock',QAS_dict['A6']+'-stock'], axis=1)
## the first 2 arguments correspond to the maximum volume a pipette can move expressed in uL
print('Run3 \n{} & {} \n'.format(QAS_dict['A5'],QAS_dict['A6']))
get_pipette_tip_count(300,1000, HBD_df_3)

Run3 
(1-butyl) triethylammonium Bromide & Hexamethonium Bromide 

 Small pipette tips: 176
 Small tipracks: 1.8
 Large pipette tips: 62
 Large tipracks: 0.65


In [20]:
HBD_df_4 = run4.drop([QAS_dict['A7']+'-stock',QAS_dict['A8']+'-stock'], axis=1)
## the first 2 arguments correspond to the maximum volume a pipette can move expressed in uL
print('Run4 \n{} & {} \n'.format(QAS_dict['A7'],QAS_dict['A8']))
get_pipette_tip_count(300,1000, HBD_df_4)

Run4 
Tetraethylammonium Iodide & Trimethylpropylammonium Bromide 

 Small pipette tips: 179
 Small tipracks: 1.9
 Large pipette tips: 58
 Large tipracks: 0.6


In [19]:
HBD_df_5 = run5.drop([QAS_dict['A9']+'-stock',QAS_dict['A10']+'-stock'], axis=1)
## the first 2 arguments correspond to the maximum volume a pipette can move expressed in uL
print('Run5 \n{} & {} \n'.format(QAS_dict['A9'],QAS_dict['A10']))
get_pipette_tip_count(300,1000, HBD_df_5)

Run5 
Ethylammonium Chloride & 2-hydroxyethylammonium Chloride 

 Small pipette tips: 72
 Small tipracks: 0.8
 Large pipette tips: 158
 Large tipracks: 1.65


___
___

In [76]:
labware_dir_path = r"./Custom Labware"
custom_labware_dict = ALH.custom_labware_dict(labware_dir_path)
# custom_labware_dict

### Step 9: Set up Loaded Dictionary and Directions for Volume Handling
It is important to know how many stock container will be needed for each stock and be able to provide the correct instructions for OT2 commands to follow.

The ranges of stock refers to the ranges of wells one stock will cover. The range is provided in terms of the index of the stock volume dataframe i.e.([lower, upper])). The format is as follows = [[stock1_range1, stock1_range2....], [stock2_range1, stock2_range2....] []...]. 

The range is based on the provided maximum volume for each container, since variabiltiy exist between filling it is **HIGHLY RECCOMENDED** to underestimate the total volume of stock a container can hold so for a 20mL scitillation vials 16000-17000 should be use as the max. Another reason for this is the OT2 will do its best to pick up as much volume as possible but understand liquid in vials can coat the sides not leaving enough depth for the OT2 to grab, so it is **HIGHLY RECCOMENDED** to have at least a **15-20%** volume buffer for each stock

The basis of range is on the maximum volume of the stock container, however currently limitation are: 
 * All stock containers must be the same

In [14]:
# Test with octadecene first, pipette.touch_tip() for stock, not sure for sample would depend on if component or sample wise making
# # then how to do every other grid, just add rows of zeros!

final_df = CreateSamples.isolate_common_column(complete_df,'stock').round(0)

protocol = execute.get_protocol_api('2.8', extra_labware=custom_labware_dict)
loaded_dict = ALH.loading_labware(protocol, plan)
stock_position_info = ALH.stock_well_ranges(complete_df, loaded_dict['Stock Wells'], volume_buffer_pct=10) 
stock_position_info

Failed to initialize character device, will not be able to control gpios (lights, button, smoothiekill, smoothie reset). Only one connection can be made to the gpios at a time. If you need to control gpios, first stop the robot server with systemctl stop opentrons-robot-server. Until you restart the server with systemctl start opentrons-robot-server, you will be unable to control the robot using the Opentrons app.


{'OleicAc-stock amount volume uL': {'Ranges': [[0, 96]],
  'Total Volume': 3372.064606741573,
  'Stock Wells': [A1 of 20mLscintillation_small 12 Well Plate 18000 µL on 1]},
 'Oleylamine-stock amount volume uL': {'Ranges': [[0, 96]],
  'Total Volume': 5922.287822878231,
  'Stock Wells': [A2 of 20mLscintillation_small 12 Well Plate 18000 µL on 1]},
 'CdAc-TOP-stock amount volume uL': {'Ranges': [[0, 96]],
  'Total Volume': 4900.0,
  'Stock Wells': [A3 of 20mLscintillation_small 12 Well Plate 18000 µL on 1]},
 'Se-TOP-stock amount volume uL': {'Ranges': [[0, 96]],
  'Total Volume': 2425.0,
  'Stock Wells': [A4 of 20mLscintillation_small 12 Well Plate 18000 µL on 1]},
 'Octadecene-stock volume uL': {'Ranges': [[0, 46], [46, 96]],
  'Total Volume': 31380.6475703802,
  'Stock Wells': [B1 of 20mLscintillation_small 12 Well Plate 18000 µL on 1,
   B2 of 20mLscintillation_small 12 Well Plate 18000 µL on 1]}}

In [15]:
Octadecene_directions = ALH.create_sample_making_directions(final_df['Octadecene-stock volume uL'], stock_position_info, loaded_dict, start_position=0)
# Octadecene_directions
directions = ALH.create_sample_making_directions(final_df.drop('Octadecene-stock volume uL', axis=1), stock_position_info, loaded_dict, start_position=0)
# directions

In [16]:
ALH.pipette_volumes_component_wise(protocol, Octadecene_directions, loaded_dict, mix_before=(1,100), new_tip='never', touch_tip = True)

Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 10
Transferring 363.0 from B1 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 to A1 of Abgene_short 96 Well Plate 2200 µL on 2
Mixing 1 times with a volume of 100.0 ul
Aspirating 100.0 uL from B1 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 50.0 uL/sec
Dispensing 100.0 uL into B1 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 300.0 uL/sec
Aspirating 181.5 uL from B1 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 50.0 uL/sec
Touching tip
Dispensing 181.5 uL into A1 of Abgene_short 96 Well Plate 2200 µL on 2 at 300.0 uL/sec
Touching tip
Mixing 1 times with a volume of 100.0 ul
Aspirating 100.0 uL from B1 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 50.0 uL/sec
Dispensing 100.0 uL into B1 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 300.0 uL/sec
Aspirating 181.5 uL from B1 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 50.0 uL/sec
Touching tip
Dispensing

Mixing 1 times with a volume of 100.0 ul
Aspirating 100.0 uL from B2 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 50.0 uL/sec
Dispensing 100.0 uL into B2 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 300.0 uL/sec
Aspirating 299.0 uL from B2 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 50.0 uL/sec
Touching tip
Dispensing 299.0 uL into E11 of Abgene_short 96 Well Plate 2200 µL on 2 at 300.0 uL/sec
Touching tip
Transferring 286.0 from B2 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 to E12 of Abgene_short 96 Well Plate 2200 µL on 2
Mixing 1 times with a volume of 100.0 ul
Aspirating 100.0 uL from B2 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 50.0 uL/sec
Dispensing 100.0 uL into B2 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 300.0 uL/sec
Aspirating 286.0 uL from B2 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 50.0 uL/sec
Touching tip
Dispensing 286.0 uL into E12 of Abgene_short 96 Well Plate 2200 µL on

In [17]:
ALH.pipette_volumes_component_wise(protocol, directions, loaded_dict, mix_before=(1,300), mix_after=(2,100),new_tip='once', blow_out=True, touch_tip = True)

Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 10
Transferring 363.0 from B1 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 to A1 of Abgene_short 96 Well Plate 2200 µL on 2
Mixing 1 times with a volume of 100.0 ul
Aspirating 100.0 uL from B1 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 50.0 uL/sec
Dispensing 100.0 uL into B1 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 300.0 uL/sec
Aspirating 181.5 uL from B1 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 50.0 uL/sec
Touching tip
Dispensing 181.5 uL into A1 of Abgene_short 96 Well Plate 2200 µL on 2 at 300.0 uL/sec
Touching tip
Mixing 1 times with a volume of 100.0 ul
Aspirating 100.0 uL from B1 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 50.0 uL/sec
Dispensing 100.0 uL into B1 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 300.0 uL/sec
Aspirating 181.5 uL from B1 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 50.0 uL/sec
Touching tip
Dispensing

Touching tip
Dispensing 193.0 uL into D7 of Abgene_short 96 Well Plate 2200 µL on 2 at 300.0 uL/sec
Touching tip
Transferring 374.0 from B1 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 to D8 of Abgene_short 96 Well Plate 2200 µL on 2
Mixing 1 times with a volume of 100.0 ul
Aspirating 100.0 uL from B1 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 50.0 uL/sec
Dispensing 100.0 uL into B1 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 300.0 uL/sec
Aspirating 187.0 uL from B1 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 50.0 uL/sec
Touching tip
Dispensing 187.0 uL into D8 of Abgene_short 96 Well Plate 2200 µL on 2 at 300.0 uL/sec
Touching tip
Mixing 1 times with a volume of 100.0 ul
Aspirating 100.0 uL from B1 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 50.0 uL/sec
Dispensing 100.0 uL into B1 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 300.0 uL/sec
Aspirating 187.0 uL from B1 of 20mLscintillation_small 12 Well Plate 

Dispensing 100.0 uL into B2 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 300.0 uL/sec
Aspirating 152.0 uL from B2 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 50.0 uL/sec
Touching tip
Dispensing 152.0 uL into G10 of Abgene_short 96 Well Plate 2200 µL on 2 at 300.0 uL/sec
Touching tip
Mixing 1 times with a volume of 100.0 ul
Aspirating 100.0 uL from B2 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 50.0 uL/sec
Dispensing 100.0 uL into B2 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 300.0 uL/sec
Aspirating 152.0 uL from B2 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 50.0 uL/sec
Touching tip
Dispensing 152.0 uL into G10 of Abgene_short 96 Well Plate 2200 µL on 2 at 300.0 uL/sec
Touching tip
Transferring 291.0 from B2 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 to G11 of Abgene_short 96 Well Plate 2200 µL on 2
Mixing 1 times with a volume of 100.0 ul
Aspirating 100.0 uL from B2 of 20mLscintillation_small 12 Well Pla

Dispensing 20.0 uL into A1 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 50.0 uL/sec
Aspirating 20.0 uL from A1 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 50.0 uL/sec
Touching tip
Dispensing 20.0 uL into D2 of Abgene_short 96 Well Plate 2200 µL on 2 at 50.0 uL/sec
Mixing 2 times with a volume of 20.0 ul
Aspirating 20.0 uL from D2 of Abgene_short 96 Well Plate 2200 µL on 2 at 50.0 uL/sec
Dispensing 20.0 uL into D2 of Abgene_short 96 Well Plate 2200 µL on 2 at 50.0 uL/sec
Aspirating 20.0 uL from D2 of Abgene_short 96 Well Plate 2200 µL on 2 at 50.0 uL/sec
Dispensing 20.0 uL into D2 of Abgene_short 96 Well Plate 2200 µL on 2 at 50.0 uL/sec
Touching tip
Blowing out at A1 of Opentrons Fixed Trash on 12
Dropping tip into A1 of Opentrons Fixed Trash on 12
Transferring 20.0 from A1 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 to D3 of Abgene_short 96 Well Plate 2200 µL on 2
Picking up tip from G3 of Opentrons 96 Tip Rack 20 µL on 11
Mixing 1 times with a vo

Dispensing 60.0 uL into F9 of Abgene_short 96 Well Plate 2200 µL on 2 at 300.0 uL/sec
Mixing 2 times with a volume of 20.0 ul
Aspirating 20.0 uL from F9 of Abgene_short 96 Well Plate 2200 µL on 2 at 50.0 uL/sec
Dispensing 20.0 uL into F9 of Abgene_short 96 Well Plate 2200 µL on 2 at 300.0 uL/sec
Aspirating 20.0 uL from F9 of Abgene_short 96 Well Plate 2200 µL on 2 at 50.0 uL/sec
Dispensing 20.0 uL into F9 of Abgene_short 96 Well Plate 2200 µL on 2 at 300.0 uL/sec
Touching tip
Blowing out at A1 of Opentrons Fixed Trash on 12
Dropping tip into A1 of Opentrons Fixed Trash on 12
Transferring 60.0 from A1 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 to F10 of Abgene_short 96 Well Plate 2200 µL on 2
Picking up tip from F4 of Opentrons 96 Tip Rack 300 µL on 10
Mixing 1 times with a volume of 20.0 ul
Aspirating 20.0 uL from A1 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 50.0 uL/sec
Dispensing 20.0 uL into A1 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 300.

...

Aspirating 50.0 uL from A3 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 50.0 uL/sec
Touching tip
Dispensing 50.0 uL into A2 of Abgene_short 96 Well Plate 2200 µL on 2 at 300.0 uL/sec
Mixing 2 times with a volume of 20.0 ul
Aspirating 20.0 uL from A2 of Abgene_short 96 Well Plate 2200 µL on 2 at 50.0 uL/sec
Dispensing 20.0 uL into A2 of Abgene_short 96 Well Plate 2200 µL on 2 at 300.0 uL/sec
Aspirating 20.0 uL from A2 of Abgene_short 96 Well Plate 2200 µL on 2 at 50.0 uL/sec
Dispensing 20.0 uL into A2 of Abgene_short 96 Well Plate 2200 µL on 2 at 300.0 uL/sec
Touching tip
Blowing out at A1 of Opentrons Fixed Trash on 12
Dropping tip into A1 of Opentrons Fixed Trash on 12
Transferring 50.0 from A3 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 to A3 of Abgene_short 96 Well Plate 2200 µL on 2
Picking up tip from C8 of Opentrons 96 Tip Rack 300 µL on 7
Mixing 1 times with a volume of 20.0 ul
Aspirating 20.0 uL from A3 of 20mLscintillation_small 12 Well Plate 18000 µL o

Aspirating 20.0 uL from D2 of Abgene_short 96 Well Plate 2200 µL on 2 at 50.0 uL/sec
Dispensing 20.0 uL into D2 of Abgene_short 96 Well Plate 2200 µL on 2 at 300.0 uL/sec
Aspirating 20.0 uL from D2 of Abgene_short 96 Well Plate 2200 µL on 2 at 50.0 uL/sec
Dispensing 20.0 uL into D2 of Abgene_short 96 Well Plate 2200 µL on 2 at 300.0 uL/sec
Touching tip
Blowing out at A1 of Opentrons Fixed Trash on 12
Dropping tip into A1 of Opentrons Fixed Trash on 12
Transferring 100.0 from A3 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 to D3 of Abgene_short 96 Well Plate 2200 µL on 2
Picking up tip from B12 of Opentrons 96 Tip Rack 300 µL on 7
Mixing 1 times with a volume of 20.0 ul
Aspirating 20.0 uL from A3 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 50.0 uL/sec
Dispensing 20.0 uL into A3 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 300.0 uL/sec
Aspirating 100.0 uL from A3 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 50.0 uL/sec
Touching tip
Dispen

Dispensing 20.0 uL into G7 of Abgene_short 96 Well Plate 2200 µL on 2 at 300.0 uL/sec
Touching tip
Blowing out at A1 of Opentrons Fixed Trash on 12
Dropping tip into A1 of Opentrons Fixed Trash on 12
Transferring 50.0 from A3 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 to G8 of Abgene_short 96 Well Plate 2200 µL on 2
Picking up tip from A4 of Opentrons 96 Tip Rack 300 µL on 4
Mixing 1 times with a volume of 20.0 ul
Aspirating 20.0 uL from A3 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 50.0 uL/sec
Dispensing 20.0 uL into A3 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 300.0 uL/sec
Aspirating 50.0 uL from A3 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 50.0 uL/sec
Touching tip
Dispensing 50.0 uL into G8 of Abgene_short 96 Well Plate 2200 µL on 2 at 300.0 uL/sec
Mixing 2 times with a volume of 20.0 ul
Aspirating 20.0 uL from G8 of Abgene_short 96 Well Plate 2200 µL on 2 at 50.0 uL/sec
Dispensing 20.0 uL into G8 of Abgene_short 96 Well Pla

Transferring 38.0 from A4 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 to C1 of Abgene_short 96 Well Plate 2200 µL on 2
Picking up tip from C7 of Opentrons 96 Tip Rack 300 µL on 4
Mixing 1 times with a volume of 20.0 ul
Aspirating 20.0 uL from A4 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 50.0 uL/sec
Dispensing 20.0 uL into A4 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 300.0 uL/sec
Aspirating 38.0 uL from A4 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 50.0 uL/sec
Touching tip
Dispensing 38.0 uL into C1 of Abgene_short 96 Well Plate 2200 µL on 2 at 300.0 uL/sec
Mixing 2 times with a volume of 20.0 ul
Aspirating 20.0 uL from C1 of Abgene_short 96 Well Plate 2200 µL on 2 at 50.0 uL/sec
Dispensing 20.0 uL into C1 of Abgene_short 96 Well Plate 2200 µL on 2 at 300.0 uL/sec
Aspirating 20.0 uL from C1 of Abgene_short 96 Well Plate 2200 µL on 2 at 50.0 uL/sec
Dispensing 20.0 uL into C1 of Abgene_short 96 Well Plate 2200 µL on 2 at 300.0 uL/s

Aspirating 20.0 uL from A4 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 50.0 uL/sec
Dispensing 20.0 uL into A4 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 300.0 uL/sec
Aspirating 25.0 uL from A4 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 at 50.0 uL/sec
Touching tip
Dispensing 25.0 uL into F4 of Abgene_short 96 Well Plate 2200 µL on 2 at 300.0 uL/sec
Mixing 2 times with a volume of 20.0 ul
Aspirating 20.0 uL from F4 of Abgene_short 96 Well Plate 2200 µL on 2 at 50.0 uL/sec
Dispensing 20.0 uL into F4 of Abgene_short 96 Well Plate 2200 µL on 2 at 300.0 uL/sec
Aspirating 20.0 uL from F4 of Abgene_short 96 Well Plate 2200 µL on 2 at 50.0 uL/sec
Dispensing 20.0 uL into F4 of Abgene_short 96 Well Plate 2200 µL on 2 at 300.0 uL/sec
Touching tip
Blowing out at A1 of Opentrons Fixed Trash on 12
Dropping tip into A1 of Opentrons Fixed Trash on 12
Transferring 38.0 from A4 of 20mLscintillation_small 12 Well Plate 18000 µL on 1 to F5 of Abgene_short 96 Well Pl

### Step 10: Format for Exporting and Upload to Google Drive
* Using the information from our previously created dataframes we create a final dataframe to convert to a csv and upload to google drive for storage. 
    * Currently the two main pieces of information uploaded are composition of sample and sample location information. This for each sample is tied to a unique ID which contains a well_timestamp_keyword. 

* Currently the function *CreateSamples.add_final_location* uses direciton information plus the sampling dataframe either created or imported to add useful information and export as a csv.

In [ ]:
filaname = ''

In [35]:
final_directions = ALH.create_sample_making_directions(run1, stock_position_info, loaded_dict, start_position=0)
export_df = CreateSamples.add_final_location(final_directions,final_df)
export_df.to_csv('./'+filename)
export_df

,UID,Labware,Slot,Well,OleicAc-stock amount volume uL,Oleylamine-stock amount volume uL,CdAc-TOP-stock amount volume uL,Se-TOP-stock amount volume uL,Octadecene-stock volume uL
480,S2_A1_09-29-2021,Abgene_short 96 Well Plate 2200 µL,2,A1,79.0,62.0,25.0,0.0,334.0
481,S2_A2_09-29-2021,Abgene_short 96 Well Plate 2200 µL,2,A2,79.0,62.0,25.0,12.0,321.0
482,S2_A3_09-29-2021,Abgene_short 96 Well Plate 2200 µL,2,A3,79.0,62.0,25.0,25.0,309.0
483,S2_A4_09-29-2021,Abgene_short 96 Well Plate 2200 µL,2,A4,79.0,62.0,25.0,38.0,296.0
484,S2_A5_09-29-2021,Abgene_short 96 Well Plate 2200 µL,2,A5,79.0,62.0,25.0,50.0,284.0
...,...,...,...,...,...,...,...,...,...
571,S2_H8_09-29-2021,Abgene_short 96 Well Plate 2200 µL,2,H8,40.0,82.0,100.0,12.0,266.0
572,S2_H9_09-29-2021,Abgene_short 96 Well Plate 2200 µL,2,H9,40.0,82.0,100.0,25.0,253.0
573,S2_H10_09-29-2021,Abgene_short 96 Well Plate 2200 µL,2,H10,40.0,82.0,100.0,38.0,241.0
574,S2_H11_09-29-2021,Abgene_short 96 Well Plate 2200 µL,2,H11,40.0,82.0,100.0,50.0,228.0
